In [94]:
import pandas as pd
from pathlib import Path

data_folder = str(Path('.') / '../../input_data/us_ca')
data_path1 = str(Path(data_folder) / 'trefferliste_2022.xls')
data_path2 = str(Path(data_folder) / 'trefferliste_2023.xls')
data_path3 = str(Path(data_folder) / 'trefferliste_2024.xls')
data_out_path = "us_data_en.csv"

In [79]:
df1 = pd.read_excel(data_path1, header=1, usecols=['Veröffentlichungs-Nummer', 'Titel', 'Zusammenfassung', 'Veröffentlichungs-Datum'])
df2 = pd.read_excel(data_path2, header=1, usecols=['Veröffentlichungs-Nummer', 'Titel', 'Zusammenfassung', 'Veröffentlichungs-Datum'])
df3 = pd.read_excel(data_path3, header=1, usecols=['Veröffentlichungs-Nummer', 'Titel', 'Zusammenfassung', 'Veröffentlichungs-Datum'])

In [80]:
# concatenate the dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)

In [81]:
df.rename(columns={'Veröffentlichungs-Nummer': 'id', 'Titel': 'title', 'Zusammenfassung': 'abstract', 'Veröffentlichungs-Datum': 'pub_date'}, inplace=True)

In [82]:
df.dropna(subset=['abstract'], inplace=True)

In [83]:
from arrangement import arrange_txt_en

In [84]:
mask_en = df['title'].str.contains(r'\[EN\]') & df['abstract'].str.contains(r'\[EN\]')
df = df[mask_en].copy()

In [85]:
df[['title', 'abstract']] = df[['title', 'abstract']].apply(arrange_txt_en, axis=1)

In [86]:
df.dropna(subset=['abstract'], inplace=True)

In [87]:
df['pub_date'] = pd.to_datetime(df['pub_date'], format='%d.%m.%Y')

In [88]:
df_filtered = df.sort_values(by='pub_date', ascending=False).drop_duplicates(subset='abstract')

In [89]:
pd.set_option('display.max_colwidth', None)

In [90]:
# change pub_date to pub_year
# Convert pub_date to datetime format
# Extract the year from pub_date to create pub_year
df_filtered['pub_year'] = df_filtered['pub_date'].dt.year
df_filtered['text'] = df_filtered['title'] + '. ' + df_filtered['abstract']

In [91]:
df_filtered[['text']].head(2)

,text
8294,"ROBOTIC ARM OPERATING SYSTEM. The robotic arm operating system includes a control device, a plurality of joint devices and a brake release monitoring device. The control device is used to generate an operation instruction. The joint device is coupled to the control device. Each joint device includes a motor and a driver. The drivers of the joint devices receive the operation instruction to generate corresponding multiple unlocking signals. The unlocking signals are used to release a braking state of the motors of the corresponding joint devices. The brake release monitoring device is coupled to the control device and the joint devices, and includes a plurality of monitoring circuits. When one of the plurality of monitoring circuits does not receive the corresponding unlocking signal, the brake release monitoring device notifies the control device that the operation instruction is not allowed."
8304,"COATING DEVICE AND CORRESPONDING COATING PROCESS. The disclosure concerns a coating device and a corresponding coating process for coating components, in particular motor vehicle body components, with a coating agent (e.g. paint), with a coating robot with a first printhead which is mounted on the coating robot. The disclosure provides that the first printhead is exchangeably mounted on the coating robot and can be exchanged for a second printhead during a color change. Another variant of the disclosure, on the other hand, provides for a second printhead to be mounted on the coating robot in addition to the first printhead, the two printheads each applying a specific coating agent in order to enable a color change without changing the printhead."


In [92]:
df_filtered.drop(columns=['title', 'abstract', 'pub_date'], inplace=True)

In [95]:
df_filtered[['id', 'pub_year', 'text']].to_csv(data_out_path, index=False)